In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
# from tensorflow.contrib import learn
import gensim
import re
from collections import namedtuple
from gensim.models import Doc2Vec
# from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from random import shuffle
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import preprocessing
import pandas as pd

max_features=1000
max_document_length=160
vocabulary=None
doc2ver_bin="smsdoc2ver.bin"
word2ver_bin="smsword2ver.bin"
SentimentDocument = namedtuple('SentimentDocument', 'words tags')

# 8.1　数据集

In [27]:
def load_all_files():
    x=[]
    y=[]
    datafile="../Data/smsspamcollection/SMSSpamCollection.txt"
    with open(datafile) as f:
        for line in f:
            line=line.strip('\n')
            label,text=line.split('\t')
            x.append(text)
            if label == 'ham':
                y.append(0)
            else:
                y.append(1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = load_all_files()

In [30]:
pd.DataFrame({"x_train":x_train,"y_train":y_train})

,x_train,y_train
0,R ü comin back for dinner?,0
1,Would you like to see my XXX pics they are so ...,1
2,Dont know you bring some food,0
3,"Haha awesome, I might need to take you up on t...",0
4,Fun fact: although you would think armand woul...,0
...,...,...
3339,Oic... I saw him too but i tot he din c me... ...,0
3340,But i'm really really broke oh. No amount is t...,0
3341,Hey Boys. Want hot XXX pics sent direct 2 ur p...,1
3342,U dun say so early hor... U c already then say...,0


# 8.2　特征提取

## 8.2.1　词袋和TF-IDF模型

### 词袋模型

In [6]:
def get_features_by_wordbag():
    global max_features
    x_train, x_test, y_train, y_test=load_all_files()

    vectorizer = CountVectorizer(
                                 decode_error='ignore',
                                 strip_accents='ascii',
                                 max_features=max_features,
                                 stop_words='english',
                                 max_df=1.0,
                                 min_df=1 )
    print(vectorizer)
    x_train=vectorizer.fit_transform(x_train)
    x_train=x_train.toarray()
    vocabulary=vectorizer.vocabulary_

    vectorizer = CountVectorizer(
                                 decode_error='ignore',
                                 strip_accents='ascii',
                                 vocabulary=vocabulary,
                                 stop_words='english',
                                 max_df=1.0,
                                 min_df=1 )
    print(vectorizer)
    x_test=vectorizer.fit_transform(x_test)
    x_test=x_test.toarray()

    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_features_by_wordbag()

CountVectorizer(decode_error='ignore', max_features=1000, stop_words='english',
                strip_accents='ascii')
CountVectorizer(decode_error='ignore', stop_words='english',
                strip_accents='ascii',
                vocabulary={'000': 0, '04': 1, '0800': 2, '08000839402': 3,
                            '08000930705': 4, '0870': 5, '08707509020': 6,
                            '08712460324': 7, '10': 8, '100': 9, '1000': 10,
                            '10am': 11, '10p': 12, '11': 13, '11mths': 14,
                            '12': 15, '12hrs': 16, '1327': 17, '150': 18,
                            '150p': 19, '150ppm': 20, '16': 21, '18': 22,
                            '1st': 23, '20': 24, '200': 25, '2000': 26,
                            '2003': 27, '20p': 28, '25': 29, ...})


In [9]:
pd.DataFrame(x_train)


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3340,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3341,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### TF-IDF模型

In [10]:
def get_features_by_wordbag_tfidf():
    global max_features
    x_train, x_test, y_train, y_test=load_all_files()

    vectorizer = CountVectorizer(
                                 decode_error='ignore',
                                 strip_accents='ascii',
                                 max_features=max_features,
                                 stop_words='english',
                                 max_df=1.0,
                                 min_df=1,
                                 binary=True)
    print(vectorizer)
    x_train=vectorizer.fit_transform(x_train)
    x_train=x_train.toarray()
    vocabulary=vectorizer.vocabulary_

    vectorizer = CountVectorizer(
                                 decode_error='ignore',
                                 strip_accents='ascii',
                                 vocabulary=vocabulary,
                                 stop_words='english',
                                 max_df=1.0,binary=True,
                                 min_df=1 )
    print(vectorizer)
    x_test=vectorizer.fit_transform(x_test)
    x_test=x_test.toarray()

    transformer = TfidfTransformer(smooth_idf=False)
    x_train=transformer.fit_transform(x_train)
    x_train=x_train.toarray()
    x_test=transformer.transform(x_test)
    x_test=x_test.toarray()

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = get_features_by_wordbag_tfidf()

CountVectorizer(binary=True, decode_error='ignore', max_features=1000,
                stop_words='english', strip_accents='ascii')
CountVectorizer(binary=True, decode_error='ignore', stop_words='english',
                strip_accents='ascii',
                vocabulary={'00': 0, '000': 1, '02': 2, '03': 3, '04': 4,
                            '06': 5, '0800': 6, '08000839402': 7,
                            '08000930705': 8, '0870': 9, '08707509020': 10,
                            '08712300220': 11, '10': 12, '100': 13, '1000': 14,
                            '10p': 15, '11': 16, '12': 17, '12hrs': 18,
                            '1327': 19, '150': 20, '150p': 21, '150ppm': 22,
                            '16': 23, '18': 24, '1st': 25, '20': 26, '200': 27,
                            '2000': 28, '2003': 29, ...})


In [11]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 8.2.2　词汇表模型

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

def get_features_by_tf():
    global max_document_length
    x_train, x_test, y_train, y_test = load_all_files()

    # 创建一个 Tokenizer 对象
    tokenizer = Tokenizer(num_words=max_document_length, oov_token="<OOV>")
    tokenizer.fit_on_texts(x_train)
    
    # 将文本转换为整数序列
    x_train_sequences = tokenizer.texts_to_sequences(x_train)
    x_test_sequences = tokenizer.texts_to_sequences(x_test)
    
    # 将序列填充到相同的长度
    x_train_padded = pad_sequences(x_train_sequences, maxlen=max_document_length, padding='post')
    x_test_padded = pad_sequences(x_test_sequences, maxlen=max_document_length, padding='post')

    return x_train_padded, x_test_padded, y_train, y_test

x_train, x_test, y_train, y_test = get_features_by_tf()

In [17]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
0,1,13,14,1,3,1,1,14,97,34,...,0,0,0,0,0,0,0,0,0,0
1,98,12,1,1,1,5,1,8,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,7,1,1,89,1,74,2,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2,68,1,1,61,1,1,2,68,26,...,0,0,0,0,0,0,0,0,0,0
4,1,39,6,1,10,1,7,1,1,25,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,107,1,10,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3340,1,93,1,103,1,1,1,3,1,3,...,0,0,0,0,0,0,0,0,0,0
3341,38,1,1,2,111,65,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
3342,1,1,8,1,1,127,39,1,1,15,...,0,0,0,0,0,0,0,0,0,0


## 8.2.3　Word2Vec模型和Doc2Vec模型

### Doc2Vec

In [37]:
def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('\n', '') for z in corpus]
    corpus = [z.replace('<br />', ' ') for z in corpus]

    # treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s ' % c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus

# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()

    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')

    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')

    return norm_text

def labelizeReviews(reviews, label_type):
    labelized = []
    for i, v in enumerate(reviews):
        label = '%s_%s' % (label_type, i)
        #labelized.append(LabeledSentence(v, [label]))
        #labelized.append(LabeledSentence(words=v,tags=label))
        labelized.append(SentimentDocument(v, [label]))
    return labelized

def getVecs(model, corpus, size):
    vecs = [np.array(model.dv[z.tags[0]]).reshape((1, size)) for z in corpus]
    return np.array(np.concatenate(vecs),dtype='float')

In [38]:
def  get_features_by_doc2vec():
    global  max_features
    x_train, x_test, y_train, y_test=load_all_files()

    x_train=cleanText(x_train)
    x_test=cleanText(x_test)

    x_train = labelizeReviews(x_train, 'TRAIN')
    x_test = labelizeReviews(x_test, 'TEST')

    x=x_train+x_test
    cores=multiprocessing.cpu_count()
    #models = [
        # PV-DBOW
    #    Doc2Vec(dm=0, dbow_words=1, size=200, window=8, min_count=19, iter=10, workers=cores),
        # PV-DM w/average
    #    Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=19, iter=10, workers=cores),
    #]
    if os.path.exists(doc2ver_bin):
        print("Find cache file %s" % doc2ver_bin)
        model=Doc2Vec.load(doc2ver_bin)
    else:
        model=Doc2Vec(dm=0, vector_size=max_features, negative=5, hs=0, min_count=2, workers=cores,epochs=60)
        model.build_vocab(x)
        model.train(x, total_examples=model.corpus_count, epochs=model.epochs)
        # model.save((doc2ver_bin)

    x_test=getVecs(model,x_test,max_features)
    x_train=getVecs(model,x_train,max_features)

    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_features_by_doc2vec()

Find cache file smsdoc2ver.bin


In [40]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.004458,-0.061380,0.004328,0.035035,0.053963,-0.035363,-0.020818,-0.003369,0.043700,0.029124,...,-0.003264,-0.042397,0.049348,-0.001347,0.089229,-0.041516,-0.015124,0.056957,-0.042357,-0.027389
1,0.070235,-0.152911,-0.094858,0.056680,0.008789,-0.030888,0.108002,-0.055896,0.005362,-0.029024,...,0.102553,-0.020463,0.076963,0.001763,0.083373,0.012041,-0.046332,-0.018365,-0.105313,-0.185344
2,0.008281,-0.041929,-0.007167,0.007217,-0.004693,0.000032,0.069734,-0.035124,0.065529,0.037060,...,0.030794,-0.087129,0.092017,0.016512,0.127887,-0.043624,-0.014526,0.042212,-0.023931,-0.013367
3,-0.001964,-0.063209,0.210218,0.022131,-0.007714,-0.055662,-0.123557,0.114349,0.117815,0.018036,...,-0.010059,-0.083258,0.000747,0.032300,0.165079,-0.004385,-0.060333,0.030876,0.066477,-0.005818
4,-0.028132,-0.084640,0.004073,-0.023565,-0.048779,-0.008749,-0.074701,-0.021888,0.090763,-0.013062,...,-0.011764,-0.026315,0.081805,0.087292,0.054000,-0.033216,0.014870,0.056211,-0.047910,-0.017038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,-0.103747,-0.158253,-0.045452,0.019096,-0.140323,0.023600,-0.069634,-0.058296,0.146667,-0.038295,...,0.057565,-0.136589,0.135482,0.157111,0.073640,-0.052858,0.033509,0.104385,-0.079987,-0.032693
3340,-0.050124,-0.074816,-0.078068,0.002041,-0.066748,-0.067258,-0.014793,-0.059085,0.051038,0.035934,...,-0.030919,-0.045536,0.075405,0.020923,0.106843,-0.018137,0.062117,0.041567,-0.164500,-0.105757
3341,-0.051853,-0.071851,-0.057251,-0.019652,-0.039451,-0.002620,-0.067998,-0.054904,0.125222,0.007532,...,0.019306,-0.108308,0.067440,0.080105,0.065208,-0.034076,-0.019555,0.106129,-0.048963,0.015892
3342,0.010649,-0.085896,-0.022093,0.093712,0.012454,-0.015241,0.023913,-0.054980,0.126428,0.063123,...,0.055459,-0.065124,0.114499,0.105787,0.119010,0.028899,0.030762,0.072743,-0.029420,0.011222


### Word2Vec

In [ ]:
import numpy as np

def buildWordVector(model, text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            # 使用方括号访问词向量
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
def  get_features_by_word2vec():
    global  max_features
    global word2ver_bin
    x_train, x_test, y_train, y_test=load_all_files()

    x_train=cleanText(x_train)
    x_test=cleanText(x_test)

    x=x_train+x_test
    cores=multiprocessing.cpu_count()

    if os.path.exists(word2ver_bin):
        print("Find cache file %s" % word2ver_bin)
        model=gensim.models.Word2Vec.load(word2ver_bin)
    else:
        model=gensim.models.Word2Vec(vector_size=max_features, window=10, min_count=1, epochs=60, workers=cores)

        model.build_vocab(x)

        model.train(x, total_examples=model.corpus_count, epochs=model.epochs)
        # model.save(word2ver_bin)

    # 将组成短信的单词和字符的Word2Vec相加并取平均值
    x_train= np.concatenate([buildWordVector(model,z, max_features) for z in x_train])
    x_train = scale(x_train)
    x_test= np.concatenate([buildWordVector(model,z, max_features) for z in x_test])
    x_test = scale(x_test)

    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = get_features_by_word2vec()

In [32]:
np.concatenate([[1],[3],[4]])

array([1, 3, 4])

In [33]:
scale(np.concatenate([[1],[3],[4]])) 

# scale 函数会将数据的每个特征（列）减去其均值并除以其标准差，从而使得每个特征的均值为 0，标准差为 1
# 使用scale函数的作用是，避免多维数据中个别维度的数据过大或者过小从而影响算法分类效果

array([-1.33630621,  0.26726124,  1.06904497])

In [26]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-0.766432,-0.085114,0.794647,1.581439,0.249115,-0.177475,1.191238,0.355237,-0.025186,-0.498360,...,-1.370850,1.265424,1.112254,1.180773,1.023409,-0.306360,1.072285,0.650548,1.017748,0.190572
1,-0.888473,1.407544,-2.178053,-1.218416,0.277493,0.934880,-0.873954,0.088817,-0.952073,-0.720517,...,2.135167,0.065429,-0.724880,-0.843104,-0.583777,0.622140,-3.013898,0.104693,0.623398,0.321471
2,-0.952926,-0.112136,-0.902263,-1.046700,1.357294,0.884033,-1.056025,-0.860502,-0.026913,0.352280,...,0.773565,1.694021,0.823311,-0.407370,0.539935,-0.665450,-0.516967,-1.283426,-0.243830,0.984675
3,0.499880,-0.568623,-0.580462,-0.554188,0.696698,0.166875,0.103681,0.368997,0.267327,-0.102114,...,0.836488,0.175721,-0.000370,1.154574,-1.202631,-0.760085,-0.238016,-0.451676,-0.447109,-0.160259
4,1.260642,3.510728,-1.663179,-0.939204,0.054370,-0.391761,2.957015,1.749006,1.609943,0.521986,...,2.346884,0.110440,-1.837250,-2.013840,1.632917,1.789204,-2.427666,-0.296104,0.941223,-1.326385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,-0.348794,0.677643,1.205271,1.694639,-0.383443,-0.520986,-0.426823,0.780282,-1.032909,-0.701918,...,-1.675168,1.880448,0.641596,1.873329,0.297619,-0.574747,0.386792,1.027732,0.825089,0.784090
3340,2.042167,-0.500722,1.688361,1.750509,1.238162,0.148662,-0.099501,0.905543,2.124442,-1.030257,...,0.638152,1.220170,0.848315,0.497110,-0.684422,1.175211,0.067510,1.597693,-0.433835,-0.157550
3341,-0.035013,0.530499,-1.157368,-0.474230,0.728233,0.936141,-1.492215,0.151246,0.750764,1.101072,...,0.294613,0.970884,0.272533,-0.471355,1.453732,0.143975,-0.069851,-0.420969,-0.624299,0.306470
3342,-0.301434,-0.153360,1.337651,0.207332,-0.323442,-1.167561,-0.362915,-1.011502,-0.479157,-0.749759,...,-0.914934,-0.650348,0.026091,-0.431718,0.049124,0.521676,1.406654,0.035817,-0.503885,-0.366673


生成的结果中有负数是正常的，这是因为词向量（word vectors）可以是多维空间中的任何实数向量，实数包括正数、负数和零。词向量是通过训练模型（如Word2Vec）学习得到的，它们捕捉了词汇的语义信息。在这些向量空间中，数值的正负并不具有直观的语义意义，但它们可以影响向量之间的相似度计算（如余弦相似度）。

关于您提供的 `DataFrame` 中每一行每一列的含义：

1. **每一行**：代表一个文档（或文本样本）的向量表示。在自然语言处理中，一个文档可以被转换成一个固定长度的向量，以便于模型处理。这个向量是通过某种方法（如词袋模型、TF-IDF 或词嵌入）将文档中的单词转换为数值表示。

2. **每一列**：代表向量中的一个维度。在词向量模型中，每个维度可以捕捉到词汇的不同语义属性。例如，一个维度可能与性别相关，另一个维度可能与情感极性相关。但是，这些维度的具体含义通常是不可解释的，因为它们是模型自动学习得到的。

在您的例子中，如果 `get_features_by_word2vec()` 函数的目的是将文档转换为词向量表示，那么：

- `x_train` 和 `x_test` 中的每一行都是一个文档的词向量表示。
- 每一列都是词向量的一个维度，这些维度共同构成了文档的高维向量表示。

如果您使用的是Word2Vec模型来生成这些向量，那么每个文档的向量表示可能是通过以下方法之一得到的：

- **平均词向量**：对于每个文档，计算其中所有单词的词向量的平均值，以得到文档的向量表示。
- **TF-IDF 加权和**：使用 TF-IDF 权重来加权每个单词的词向量，然后求和得到文档的向量表示。
- **其他方法**：如Doc2Vec，它直接学习文档的向量表示。

请注意，具体的转换方法取决于您的 `get_features_by_word2vec()` 函数的实现细节。


### ngram

In [12]:
def get_features_by_ngram():
    global max_features
    x_train, x_test, y_train, y_test=load_all_files()

    vectorizer = CountVectorizer(
                                 decode_error='ignore',
                                 ngram_range=(3, 3),
                                 strip_accents='ascii',
                                 max_features=max_features,
                                 stop_words='english',
                                 max_df=1.0,
                                 min_df=1,
                                 token_pattern=r'\b\w+\b',
                                 binary=True)
    print(vectorizer)
    x_train=vectorizer.fit_transform(x_train)
    x_train=x_train.toarray()
    vocabulary=vectorizer.vocabulary_

    vectorizer = CountVectorizer(
                                 decode_error='ignore',
                                 ngram_range=(3, 3),
                                 strip_accents='ascii',
                                 vocabulary=vocabulary,
                                 stop_words='english',
                                 max_df=1.0,binary=True,
                                 token_pattern=r'\b\w+\b',
                                 min_df=1 )
    print(vectorizer)
    x_test=vectorizer.fit_transform(x_test)
    x_test=x_test.toarray()

    transformer = TfidfTransformer(smooth_idf=False)
    x_train=transformer.fit_transform(x_train)
    x_train=x_train.toarray()
    x_test=transformer.transform(x_test)
    x_test=x_test.toarray()

    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = get_features_by_ngram()

CountVectorizer(binary=True, decode_error='ignore', max_features=1000,
                ngram_range=(3, 3), stop_words='english', strip_accents='ascii',
                token_pattern='\\b\\w+\\b')
CountVectorizer(binary=True, decode_error='ignore', ngram_range=(3, 3),
                stop_words='english', strip_accents='ascii',
                token_pattern='\\b\\w+\\b',
                vocabulary={'000 bonus caller': 0, '000 cash await': 1,
                            '02 06 03': 2, '03 2nd attempt': 3, '06 03 2nd': 4,
                            '08702840625 comuk 220': 5,
                            '08707509020 just 20p': 6,
                            '08712300220 quoting claim': 7,
                            '087187262701 50gbp mtmsg18': 8,
                            '09050090044 toclaim s...9,
                            '09061209465 c suprman': 10,
                            '09061221066 fromm landline': 11, '1 2 3': 12,
                            '1 2 price': 13, '1 50 msg'

In [13]:
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


# 8.3　模型训练与验证

## 8.3.1　朴素贝叶斯算法

In [44]:
def do_nb_wordbag(x_train, x_test, y_train, y_test):
    print("NB and wordbag")
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_nb_doc2vec(x_train, x_test, y_train, y_test):
    print("NB and doc2vec")
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_nb_word2vec(x_train, x_test, y_train, y_test):
    print("NB and word2vec")
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [45]:

def do_nb_doc2vec(x_train, x_test, y_train, y_test):
    print("NB and doc2vec")
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_nb_word2vec(x_train, x_test, y_train, y_test):
    print("NB and word2vec")
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    y_pred=gnb.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

## 8.3.2　支持向量机算法

In [46]:
def do_svm_wordbag(x_train, x_test, y_train, y_test):
    print("SVM and wordbag")
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))


def do_svm_doc2vec(x_train, x_test, y_train, y_test):
    print("SVM and doc2vec")
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_svm_word2vec(x_train, x_test, y_train, y_test):
    print("SVM and word2vec")
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

## 8.3.3　XGBoost算法

In [47]:
def do_xgboost_word2vec(x_train, x_test, y_train, y_test):
    print("xgboost and word2vec")
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_xgboost_wordbag(x_train, x_test, y_train, y_test):
    print("xgboost and wordbag")
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

## 8.3.4　深度学习算法之MLP

In [48]:
def do_dnn_wordbag(x_train, x_test, y_train, y_test):
    print("MLP and wordbag")

    # Building deep neural network
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (5, 2),
                        random_state = 1)
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_dnn_doc2vec(x_train, x_test, y_train, y_test):
    print("MLP and doc2vec")
    global max_features
    # Building deep neural network
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (5, 2),
                        random_state = 1)
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_dnn_word2vec(x_train, x_test, y_train, y_test):
    print("MLP and word2vec")
    global max_features
    # Building deep neural network
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (5, 2),
                        random_state = 1)
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_dnn_wordbag(x_train, x_test, y_train, y_test):
    print("MLP and wordbag")
    global max_features
    # Building deep neural network
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (5, 2),
                        random_state = 1)
    print(clf)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

## 随机森林

In [ ]:
def do_rf_doc2vec(x_train, x_test, y_train, y_test):
    print("rf and doc2vec")
    clf = RandomForestClassifier(n_estimators=10)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))

def do_rf_word2vec(x_train, x_test, y_train, y_test):
    print("rf and word2vec")
    clf = RandomForestClassifier(n_estimators=50)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(metrics.accuracy_score(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))


## CNN

In [67]:
def do_cnn_wordbag(trainX, testX, trainY, testY):
    global max_document_length
    print("CNN and tf")

    trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
    testX = pad_sequences(testX, maxlen=max_document_length, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Building convolutional network
    network = input_data(shape=[None,max_document_length], name='input')
    network = tflearn.embedding(network, input_dim=1000000, output_dim=128)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=100,run_id="review")

In [68]:

def do_cnn_doc2vec_2d(trainX, testX, trainY, testY):
    print("CNN and doc2vec 2d")

    trainX = trainX.reshape([-1, max_features, max_document_length, 1])
    testX = testX.reshape([-1, max_features, max_document_length, 1])


    # Building convolutional network
    network = input_data(shape=[None, max_features, max_document_length, 1], name='input')
    network = conv_2d(network, 16, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = fully_connected(network, 128, activation='tanh')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='tanh')
    network = dropout(network, 0.8)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')

    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit({'input': trainX}, {'target': trainY}, n_epoch=20,
               validation_set=({'input': testX}, {'target': testY}),
               snapshot_step=100, show_metric=True, run_id='review')

In [69]:
def do_cnn_word2vec_2d(trainX, testX, trainY, testY):
    global max_features
    global max_document_length
    print("CNN and word2vec2d")
    y_test = testY
    #trainX = pad_sequences(trainX, maxlen=max_features, value=0.)
    #testX = pad_sequences(testX, maxlen=max_features, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Building convolutional network
    network = input_data(shape=[None,max_document_length,max_features,1], name='input')

    network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = fully_connected(network, 128, activation='tanh')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='tanh')
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')

    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True,run_id="sms")

    y_predict_list = model.predict(testX)
    print(y_predict_list)

    y_predict = []
    for i in y_predict_list:
        print(i[0])
        if i[0] > 0.5:
            y_predict.append(0)
        else:
            y_predict.append(1)

    print(classification_report(y_test, y_predict))
    print(metrics.confusion_matrix(y_test, y_predict))

In [71]:
def  get_features_by_word2vec_cnn_1d():
    global  max_features
    global word2ver_bin
    x_train, x_test, y_train, y_test=load_all_files()

    x_train=cleanText(x_train)
    x_test=cleanText(x_test)

    x=x_train+x_test
    cores=multiprocessing.cpu_count()

    if os.path.exists(word2ver_bin):
        print("Find cache file %s" % word2ver_bin)
        model=gensim.models.Word2Vec.load(word2ver_bin)
    else:
        model=gensim.models.Word2Vec(size=max_features, window=10, min_count=1, iter=60, workers=cores)

        model.build_vocab(x)

        model.train(x, total_examples=model.corpus_count, epochs=model.iter)
        # model.save((word2ver_bin)

    min_max_scaler = preprocessing.MinMaxScaler()

    x_train= np.concatenate([buildWordVector(model,z, max_features) for z in x_train])
    x_train = min_max_scaler.fit_transform(x_train)
    x_test= np.concatenate([buildWordVector(model,z, max_features) for z in x_test])
    x_test = min_max_scaler.transform(x_test)

    return x_train, x_test, y_train, y_test

In [70]:
def  get_features_by_word2vec_cnn_2d():
    global max_features
    global max_document_length
    global word2ver_bin

    x_train, x_test, y_train, y_test=load_all_files()

    x_train_vecs=[]
    x_test_vecs=[]

    x_train=cleanText(x_train)
    x_test=cleanText(x_test)

    x=x_train+x_test
    cores=multiprocessing.cpu_count()

    if os.path.exists(word2ver_bin):
        print("Find cache file %s" % word2ver_bin)
        model=gensim.models.Word2Vec.load(word2ver_bin)
    else:
        model=gensim.models.Word2Vec(size=max_features, window=10, min_count=1, iter=60, workers=cores)

        model.build_vocab(x)

        model.train(x, total_examples=model.corpus_count, epochs=model.iter)
        # model.save((word2ver_bin)


    #x_train_vec=np.zeros((max_document_length,max_features))
    #x_test_vec=np.zeros((max_document_length, max_features))
    """
    x_train= np.concatenate([buildWordVector(model,z, max_features) for z in x_train])
    x_train = min_max_scaler.fit_transform(x_train)
    x_test= np.concatenate([buildWordVector(model,z, max_features) for z in x_test])
    x_test = min_max_scaler.transform(x_test)
    vec += imdb_w2v[word].reshape((1, size))
    """
    #x_train = np.concatenate([buildWordVector_2d(model, z, max_features) for z in x_train])
    x_all=np.zeros((1,max_features))
    for sms in x_train:
        sms=sms[:max_document_length]
        #print sms
        x_train_vec = np.zeros((max_document_length, max_features))
        for i,w in enumerate(sms):
            vec=model[w].reshape((1, max_features))
            x_train_vec[i-1]=vec.copy()
            #x_all=np.concatenate((x_all,vec))
        x_train_vecs.append(x_train_vec)
        #print x_train_vec.shape
    for sms in x_test:
        sms=sms[:max_document_length]
        #print sms
        x_test_vec = np.zeros((max_document_length, max_features))
        for i,w in enumerate(sms):
            vec=model[w].reshape((1, max_features))
            x_test_vec[i-1]=vec.copy()
            #x_all.append(vec)
        x_test_vecs.append(x_test_vec)

    #print x_train
    #print x_all
    min_max_scaler = preprocessing.MinMaxScaler()
    print("fix min_max_scaler")
    x_train_2d=np.concatenate([z for z in x_train_vecs])
    min_max_scaler.fit(x_train_2d)

    x_train=np.concatenate([min_max_scaler.transform(i) for i in x_train_vecs])
    x_test=np.concatenate([min_max_scaler.transform(i) for i in x_test_vecs])

    x_train=x_train.reshape([-1, max_document_length, max_features, 1])
    x_test = x_test.reshape([-1, max_document_length, max_features, 1])

    return x_train, x_test, y_train, y_test

In [75]:
def do_cnn_word2vec_2d_345(trainX, testX, trainY, testY):
    global max_features
    global max_document_length
    print("CNN and word2vec_2d_345")
    y_test = testY

    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Building convolutional network
    network = input_data(shape=[None,max_document_length,max_features,1], name='input')
    network = tflearn.embedding(network, input_dim=1, output_dim=128,validate_indices=False)
    branch1 = conv_2d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_2d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_2d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool_2d(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=100,run_id="sms")

    y_predict_list = model.predict(testX)
    print(y_predict_list)

    y_predict = []
    for i in y_predict_list:
        print(i[0])
        if i[0] > 0.5:
            y_predict.append(0)
        else:
            y_predict.append(1)

    print(classification_report(y_test, y_predict))
    print(metrics.confusion_matrix(y_test, y_predict))

In [76]:
def do_cnn_word2vec(trainX, testX, trainY, testY):
    global max_features
    print("CNN and word2vec")
    y_test = testY
    #trainX = pad_sequences(trainX, maxlen=max_features, value=0.)
    #testX = pad_sequences(testX, maxlen=max_features, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Building convolutional network
    network = input_data(shape=[None,max_features], name='input')
    network = tflearn.embedding(network, input_dim=1000000, output_dim=128,validate_indices=False)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=100,run_id="sms")

    y_predict_list = model.predict(testX)
    print(y_predict_list)

    y_predict = []
    for i in y_predict_list:
        print(i[0])
        if i[0] > 0.5:
            y_predict.append(0)
        else:
            y_predict.append(1)

    print(classification_report(y_test, y_predict))
    print(metrics.confusion_matrix(y_test, y_predict))

In [77]:
def do_cnn_doc2vec(trainX, testX, trainY, testY):
    global max_features
    print("CNN and doc2vec")

    #trainX = pad_sequences(trainX, maxlen=max_features, value=0.)
    #testX = pad_sequences(testX, maxlen=max_features, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Building convolutional network
    network = input_data(shape=[None,max_features], name='input')
    network = tflearn.embedding(network, input_dim=1000000, output_dim=128,validate_indices=False)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=100,run_id="review")

## RNN

In [78]:

def do_rnn_wordbag(trainX, testX, trainY, testY):
    global max_document_length
    print("RNN and wordbag")
    y_test=testY
    trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
    testX = pad_sequences(testX, maxlen=max_document_length, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Network building
    net = tflearn.input_data([None, max_document_length])
    net = tflearn.embedding(net, input_dim=10240000, output_dim=128)
    net = tflearn.lstm(net, 128, dropout=0.8)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                             loss='categorical_crossentropy')

    # Training
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True,
              batch_size=10,run_id="sms",n_epoch=5)

    y_predict_list = model.predict(testX)
    print(y_predict_list)

    y_predict = []
    for i in y_predict_list:
        print(i[0])
        if i[0] > 0.5:
            y_predict.append(0)
        else:
            y_predict.append(1)

    print(classification_report(y_test, y_predict))
    print(metrics.confusion_matrix(y_test, y_predict))

In [79]:

def do_rnn_word2vec(trainX, testX, trainY, testY):
    global max_features
    print("RNN and wordbag")

    trainX = pad_sequences(trainX, maxlen=max_features, value=0.)
    testX = pad_sequences(testX, maxlen=max_features, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Network building
    net = tflearn.input_data([None, max_features])
    net = tflearn.embedding(net, input_dim=10240000, output_dim=128)
    net = tflearn.lstm(net, 128, dropout=0.8)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                             loss='categorical_crossentropy')

    # Training
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True,
              batch_size=10,run_id="sms",n_epoch=5)

# 集中比较


## doc2vec

In [42]:
x_train, x_test, y_train, y_test = get_features_by_doc2vec()

In [55]:
do_nb_doc2vec(x_train, x_test, y_train, y_test)

NB and doc2vec
0.9672645739910314
[[1871   53]
 [  20  286]]


In [61]:
do_rf_doc2vec(x_train, x_test, y_train, y_test)

rf and doc2vec
0.9775784753363229
[[1922    2]
 [  48  258]]


In [59]:
do_svm_doc2vec(x_train, x_test, y_train, y_test)

SVM and doc2vec
0.9838565022421525
[[1919    5]
 [  31  275]]


In [57]:
do_dnn_doc2vec(x_train, x_test, y_train, y_test)

MLP and doc2vec
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')
0.973542600896861
[[1895   29]
 [  30  276]]


In [56]:
do_cnn_doc2vec_2d(x_train, x_test, y_train, y_test)

CNN and doc2vec 2d


ValueError: cannot reshape array of size 3344000 into shape (1000,160,1)

## word2vec

In [62]:
x_train, x_test, y_train, y_test=get_features_by_word2vec()

In [63]:
do_nb_word2vec(x_train, x_test, y_train, y_test)

NB and word2vec
0.9726457399103139
[[1892   38]
 [  23  277]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1930
           1       0.88      0.92      0.90       300

    accuracy                           0.97      2230
   macro avg       0.93      0.95      0.94      2230
weighted avg       0.97      0.97      0.97      2230



In [82]:
do_dnn_word2vec(x_train, x_test, y_train, y_test)

MLP and word2vec
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1930
           1       0.91      0.94      0.93       300

    accuracy                           0.98      2230
   macro avg       0.95      0.96      0.96      2230
weighted avg       0.98      0.98      0.98      2230

[[1903   27]
 [  18  282]]


In [83]:
do_svm_word2vec(x_train, x_test, y_train, y_test)

SVM and word2vec
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1930
           1       0.99      0.93      0.96       300

    accuracy                           0.99      2230
   macro avg       0.99      0.96      0.98      2230
weighted avg       0.99      0.99      0.99      2230

0.9896860986547085
[[1928    2]
 [  21  279]]


In [84]:
do_rf_word2vec(x_train, x_test, y_train, y_test)

rf and word2vec
0.9825112107623318
[[1928    2]
 [  37  263]]


In [85]:
do_xgboost_word2vec(x_train, x_test, y_train, y_test)

xgboost and word2vec
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1930
           1       0.98      0.92      0.95       300

    accuracy                           0.99      2230
   macro avg       0.98      0.96      0.97      2230
weighted avg       0.99      0.99      0.99      2230

[[1925    5]
 [  25  275]]


In [86]:
do_rnn_word2vec(x_train, x_test, y_train, y_test)

RNN and wordbag
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


ResourceExhaustedError: OOM when allocating tensor with shape[10240000,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node Embedding_1/W/Initializer/truncated_normal/TruncatedNormal (defined at d:\Anaconda\lib\site-packages\tflearn\variables.py:63) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'Embedding_1/W/Initializer/truncated_normal/TruncatedNormal':
  File "d:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\Anaconda\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "d:\Anaconda\lib\site-packages\ipykernel\kernela

In [87]:
x_train, x_test, y_train, y_test = get_features_by_word2vec_cnn_1d()
do_cnn_word2vec(x_train, x_test, y_train, y_test)

Find cache file smsword2ver.bin
CNN and word2vec


InvalidArgumentError: Cannot parse tensor from proto: dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 1000000
  }
  dim {
    size: 128
  }
}
float_val: 0

	 [[node Embedding_2/W/Adam/Initializer/zeros (defined at d:\Anaconda\lib\site-packages\tflearn\helpers\trainer.py:716) ]]

Original stack trace for 'Embedding_2/W/Adam/Initializer/zeros':
  File "d:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\Anaconda\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "d:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "d:\Anaconda\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "d:\Anac

In [ ]:
x_train, x_test, y_train, y_test = get_features_by_word2vec_cnn_2d()
do_cnn_word2vec_2d(x_train, x_test, y_train, y_test)

CNN and word2vec2d


IndexError: list index out of range

In [ ]:
x_train, x_test, y_train, y_test = get_features_by_word2vec_cnn_1d_log()
do_cnn_word2vec_2d_345(x_train, x_test, y_train, y_test)

NameError: name 'do_cnn_word2vec_2d_345' is not defined

## wordbag

In [88]:
x_train, x_test, y_train, y_test = get_features_by_wordbag()

CountVectorizer(decode_error='ignore', max_features=1000, stop_words='english',
                strip_accents='ascii')
CountVectorizer(decode_error='ignore', stop_words='english',
                strip_accents='ascii',
                vocabulary={'00': 0, '000': 1, '02': 2, '03': 3, '04': 4,
                            '0800': 5, '08000839402': 6, '08000930705': 7,
                            '08718720201': 8, '10': 9, '100': 10, '1000': 11,
                            '10p': 12, '11': 13, '11mths': 14, '12': 15,
                            '12hrs': 16, '150': 17, '150p': 18, '150ppm': 19,
                            '16': 20, '18': 21, '1st': 22, '20': 23, '200': 24,
                            '2000': 25, '2003': 26, '2004': 27, '250': 28,
                            '25p': 29, ...})


In [89]:
do_nb_wordbag(x_train, x_test, y_train, y_test)

NB and wordbag
              precision    recall  f1-score   support

           0       0.99      0.76      0.86      1928
           1       0.38      0.94      0.55       302

    accuracy                           0.79      2230
   macro avg       0.69      0.85      0.70      2230
weighted avg       0.91      0.79      0.82      2230

[[1474  454]
 [  18  284]]


In [90]:
do_dnn_wordbag(x_train, x_test, y_train, y_test)

MLP and wordbag
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1928
           1       0.94      0.91      0.92       302

    accuracy                           0.98      2230
   macro avg       0.96      0.95      0.96      2230
weighted avg       0.98      0.98      0.98      2230

[[1909   19]
 [  26  276]]


In [ ]:
do_xgboost_wordbag(x_train, x_test, y_train, y_test)

xgboost and wordbag
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1928
           1       0.95      0.86      0.90       302

    accuracy                           0.97      2230
   macro avg       0.96      0.93      0.94      2230
weighted avg       0.97      0.97      0.97      2230

[[1914   14]
 [  42  260]]


: 

In [ ]:
do_rnn_wordbag(x_train, x_test, y_train, y_test)

RNN and wordbag


## ngram

In [ ]:
x_train, x_test, y_train, y_test = get_features_by_ngram()

## tf

In [ ]:
x_train, x_test, y_train, y_test =get_features_by_tf()
x_train, x_test, y_train, y_test=get_features_by_wordbag_tfidf()